<a href="https://colab.research.google.com/github/hoangtheanhhp/ZaloQA/blob/bert_lstm/QASystem/run_zalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

TensorFlow 1.x selected.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!test -d zalo || git clone https://github.com/hoangtheanhhp/ZaloQA zalo
import sys
if 'zalo' not in sys.path:
  sys.path+=['zalo']
%tensorflow_version 1.x
!pip install tensorflow==1.15.2
!pip install -r zalo/requirements.txt
!pip install kashgari==1.1.4

DATASET_PATH = "zalo/Dataset/zalo"
SAVE_MODEL_PATH = "/content/drive/My Drive/ZaloAI/result/model.h5"
MODEL_PATH = "/content/drive/My Drive/ZaloAI/bert"

TensorFlow 1.x selected.


In [5]:
from zalo.QASystem.preprocess import ZaloDatasetProcessor
import kashgari
from kashgari.embeddings import BERTEmbedding

bert_embedding = BERTEmbedding(MODEL_PATH,
                               task=kashgari.CLASSIFICATION,
                               sequence_length=128)

tokenizer = bert_embedding.tokenizer
processor = ZaloDatasetProcessor()
processor.load_from_path(DATASET_PATH, "train")
processor.load_from_path(DATASET_PATH, "val")

data = processor.train_data
val = processor.val_data
print(val)
train_x = []
train_y = []
val_x = []
val_y = []
for d in data:
  train_x.append(tokenizer.tokenize(d.get('question')) + 
                 tokenizer.tokenize(d.get('text'))[1:])
  train_y.append(d.get('label', False))

for d in val:
  val_x.append(tokenizer.tokenize(d.get('question')) + 
               tokenizer.tokenize(d.get('text'))[1:])
  val_y.append(d.get('label', False))

print(len(train_x), len(train_y), len(val_x), len(val_y))


100%|██████████| 14200/14200 [00:00<00:00, 925405.79it/s]
0it [00:00, ?it/s]


[]


KeyboardInterrupt: ignored

In [4]:
from tensorflow.python import keras
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.callbacks import EvalCallBack

import logging
logging.basicConfig(level='DEBUG')

model = BiLSTM_Model()


model.fit(train_x,
          train_y,
          val_x,
          val_y,
          epochs=10, batch_size=100)

model.save(SAVE_MODEL_PATH)


14200 14200 0 0
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 176)]             0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 176, 100)          620200    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 855,210
Trainable params: 855,210
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
  9/143 [>.............................] - ETA: 4:51 - loss: 0.6639 - acc: 0.6489

KeyboardInterrupt: ignored

In [14]:
processor.load_from_path(DATASET_PATH,"test")
test = processor.test_data

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(test_x, test_y, verbose=1)
print("loss: %f\n accruracy: %f\nf1_score: %f\nprecision: %f\n, recall: %f\n:" 
      % (loss, accurany,f1_score,precision, recall))

NameError: ignored